# 第4回 | おすすめのDataFrame作成方法

今回の講義では、Pandas.DataFrame作成でおすすめの方法を紹介していきます。

<br>

というのも、前回の講義でDataFrameの作成方法は3つあると解説しました。それが以下の3つです。

- パターン① : 二次元リストから作成
- パターン② : 辞書から作成
- パターン③ : 辞書を格納したリストから作成

このように作成方法はたくさんあるのですが、**本当に使っていくべきパターンは1つに限定できます。**

もちろん自分の書きやすい方法を確立したり、会社のコードスタイルに合わせたりするのが第一優先です。

<br>

でも、これからPandasを勉強していくなら、学習教材側で決め打ちになっていた方が理解が進みやすくなります。

3つの方法を一気に習得しようとするより、1つの方法だけを「着実に」身につけた方が良いってことですね。

<br>

なので、今回はおすすめのDataFrame作成方法と、その方法を選ぶ理由について紹介していきます。

今回は「APIから取得したデータをDataFrameに変換する」という、実務に近い形式で学べるようになっています。

データ分析職に就きたいと考えているなら、ぜひ学習を進めていきましょう。(もちろんそうでなくても...！)

## ライブラリのインポート

まずは、Pythonライブラリをインポートしていきましょう。

前回使っていた`Pandas`と`Numpy`だけでなく、APIのアクセスで必要な`requests`ライブラリもインポートしていきます。

<br>

そのために、まずは`requests`ライブラリをインストールしていきましょう。

そのためにはターミナルを開いて...、と言いたいところですが、実はJupyterのセルからでもライブラリのインストールが可能です。

Jupyterのセルで`requests`ライブラリをインストールするには、以下のコマンドを実行してあげます。

In [3]:
# requestsライブラリをインストールする
!conda install requests -y

Solving environment: done

## Package Plan ##

  environment location: /Users/hayato/miniforge3/envs/analysis

  added / updated specs:
    - requests


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    brotlipy-0.7.0             |py38h30f7421_1001         340 KB  conda-forge
    chardet-4.0.0              |   py38h10201cd_1         205 KB  conda-forge
    cryptography-3.4.7         |   py38h3c0dae5_0         797 KB  conda-forge
    pysocks-1.7.1              |   py38h10201cd_3          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.3 MB

The following NEW packages will be INSTALLED:

  brotlipy           conda-forge/osx-arm64::brotlipy-0.7.0-py38h30f7421_1001
  chardet            conda-forge/osx-arm64::chardet-4.0.0-py38h10201cd_1
  cryptography       conda-forge/osx-arm64::cryptography-3.4.7-

これで`requests`ライブラリのインストールが完了しました。

それでは、このままライブラリをインポートしましょう！

In [4]:
# ライブラリのインポート
import pandas as pd
import numpy as np

import requests

これでライブラリのインポートは完了しました。

それでは講義内容に入っていきます！

# Pandas.DataFrame : おすすめの作成方法

Pandas.DataFrameの作成方法は、以下の3つあります。

- パターン① : 二次元リストから作成
- パターン② : 辞書から作成
- パターン③ : 辞書を格納したリストから作成

このうち、おすすめのDataFrame作成方法は「**パターン③ : 辞書を格納したリストから作成**」です。

その理由は、2つほどあります。

- DataFrameの基になるリストを作成するとき、コードを見ただけでどのようなカラムが作成されるのか分かりやすい
- APIで取得したデータをDataFrameに変換するとき、取得したデータをブチ込むだけでDataFrameを作成できる

と理由を書いてみましたが、これだけだと何を言っているのか分かりません。それぞれを詳しく見ていきましょう！

## アクセス先のAPIについて

今回は技術情報がたくさん掲載されている[Qiita(キータ)](https://qiita.com/)のAPIで取得した記事情報を基に、DataFrameを作成してみたいと思います。

»[参考記事 : Qiita API v2ドキュメント - Qiita:Developer](https://qiita.com/api/v2/docs)

*※APIのアクセスは大事なスキルですが、Pandasと関係ないので理解できていなくても大丈夫です！*

<br>

Pythonを使ってAPIやWebサイトにアクセスするには、先ほどインポートしておいた`requests`ライブラリを使ってあげると簡単に実装できます。

Qiitaの記事情報を取得するには、以下のURLにアクセスします。

[https://qiita.com/api/v2/items](https://qiita.com/api/v2/items)

<br>

また、デフォルトではトップ20件の記事を取得するようになっています。

今回はもう少しデータを取得したいので、20件ではなく100件にパラメータ設定して、APIにアクセスしたいと思います。

コードに落とし込むなら、以下のようになります。

In [5]:
# 100件のデータを取得するようにパラメータ設定
params = {
    'per_page': 100
}

# APIにアクセスした結果を変数rに格納する
r = requests.get('https://qiita.com/api/v2/items', params=params)

これでQiita APIへのアクセスが完了しました。

あとは取得したデータを基に、DataFrameを作成していきたいと思います。

## おすすめポイント① : 分かりやすさ

パターン③の方法をおすすめする1つ目の理由が「分かりやすさ」です。

今回はQiita APIで取得した結果から、以下の項目でDataFrameを作成しましょう。

- 記事タイトル
- 記事URL
- 記事の作成日
- 記事に付いているタグ情報

そのためのコードは、以下のようになります。

In [10]:
# DataFrameの基になるリストを作成する
d_list = []
for article in r.json():
    d_list.append({
        'title': article['title'],
        'url': article['url'],
        'updated_at': article['updated_at'],
        'tags': [tag['name'] for tag in article['tags']]
    })

# DataFrameを作成して、変数dfに格納する
df = pd.DataFrame(d_list)

これでDataFrameを作成できました。

パターン③の方法だと、このようにリストを作成するコードを見るだけで、どのようなカラムを持ったDataFrameになるのか分かります。

<br>

念のため、いま作成したDataFrameの中身を確認してみましょう。

In [7]:
# DataFrameの中身を確認する
df

,title,url,updated_at,tags
0,【C++】時刻計算を簡単に行えるライブラリを自作した,https://qiita.com/NA_simple/items/ec88eda847eb...,2021-05-05T11:38:51+09:00,"[C++, datetime, OSS, MITライセンス]"
1,VBAで作るファイルリスト,https://qiita.com/kekosh/items/97083ead5442c97...,2021-05-05T11:38:09+09:00,"[メモ, VBA, 初心者, 備忘録]"
2,プログラマ育成のためにIPA風問題を作って社内勉強会で使ってみた話,https://qiita.com/remix-yh/items/fd4cd44efce5c...,2021-05-05T11:32:28+09:00,"[勉強会, プログラマ]"
3,ESP-IDFでスレーブのレジスタアドレスを指定したreadのやり方,https://qiita.com/badmintoncryer/items/752f8fe...,2021-05-05T11:21:01+09:00,"[C, I2C, ESP-IDF]"
4,小ネタ：年度とかの計算｜Power Query,https://qiita.com/tanuki_phoenix/items/746df2d...,2021-05-05T11:18:24+09:00,"[Excel, PowerBI, PowerQuery]"
...,...,...,...,...
95,STM32L010のシリアルでHello Worldと出力,https://qiita.com/caa45040/items/3e5d7bab62405...,2021-05-05T00:58:29+09:00,[STM32L010]
96,Raspberry Pi Picoでモータを動かす③（PIOで動かす①）,https://qiita.com/fmax/items/f4ae62f393bc4fc5b49a,2021-05-05T00:58:25+09:00,[RaspberryPiPico]
97,【Kotlin】unsigned integers/inline class引数のコンパイル...,https://qiita.com/wrongwrong/items/46270820077...,2021-05-05T11:27:13+09:00,"[Kotlin, reflection]"
98,Bash/Zsh/PowerShell 上の Git でマージ済みのブランチを一括削除,https://qiita.com/watagashi/items/949b58b47132...,2021-05-05T00:50:38+09:00,[Git]


上記の出力結果を確認してみると、リストを作成していたときに見ていたKeyが、DataFrameのカラム名になっていますね！

## おすすめポイント② : APIの取得結果から簡単に作成できる

また、特定の情報からDataFrameを作成するのではなく、取得結果全体を使う場合であれば、より簡潔なコードで済みます。

具体的には、取得結果`r.json()`を`pd.Dataframe()`に突っ込むだけです。

In [8]:
# APIの取得結果を基にDataFrameを作成する
df2 = pd.DataFrame(r.json())

このように、取得結果をそのままDataFrameにしたいと思ったとき、パターン③の方法であればたった1行のコードでDataFrameを作成できます。

中身を確認してみると、しっかりDataFrameを作成できていることが分かりますね。

In [9]:
# DataFrameの中身を確認する
df2

,rendered_body,body,coediting,comments_count,created_at,group,id,likes_count,private,reactions_count,tags,title,updated_at,url,user,page_views_count,team_membership
0,"\n<h2>\n<span id=""はじめに"" class=""fragment""></spa...",## はじめに\nC++ での時刻計算では、`struct tm`を使った方法がよく紹介され...,False,0,2021-05-05T11:38:51+09:00,None,ec88eda847eb1dd8f8b7,0,False,0,"[{'name': 'C++', 'versions': []}, {'name': 'da...",【C++】時刻計算を簡単に行えるライブラリを自作した,2021-05-05T11:38:51+09:00,https://qiita.com/NA_simple/items/ec88eda847eb...,{'description': '主に、機械学習の実装、Webアプリの開発を行っています。 ...,None,None
1,"\n<h1>\n<span id=""tldl"" class=""fragment""></spa...",#TL;DL\n業務で使えるのはVBAかVBScriptのみの環境で、ツールを作って作業を効...,False,0,2021-05-05T11:38:09+09:00,None,97083ead5442c975fc01,0,False,0,"[{'name': 'メモ', 'versions': []}, {'name': 'VBA...",VBAで作るファイルリスト,2021-05-05T11:38:09+09:00,https://qiita.com/kekosh/items/97083ead5442c97...,{'description': '今年３５歳になる予定の窓際業務系SEです。 嫌いな言葉は...,None,None
2,"\n<h1>\n<span id=""この記事の内容"" class=""fragment""></...",# この記事の内容\nこの記事は、社内の勉強会でIPA風の問題を作って、実際に解いてもらった...,False,0,2021-05-05T11:32:28+09:00,None,fd4cd44efce5c5a834ac,0,False,0,"[{'name': '勉強会', 'versions': []}, {'name': 'プロ...",プログラマ育成のためにIPA風問題を作って社内勉強会で使ってみた話,2021-05-05T11:32:28+09:00,https://qiita.com/remix-yh/items/fd4cd44efce5c...,{'description': 'SIerで10年ほどシステムエンジニアをやってます。最近は...,None,None
3,"\n<h1>\n<span id=""概要"" class=""fragment""></span>...",# 概要\nesp-idfを用いてI2C通信しようとしたところかなり時間を食ったので、その備...,False,0,2021-05-05T11:21:01+09:00,None,752f8fe2e5b3578ec7e8,0,False,0,"[{'name': 'C', 'versions': []}, {'name': 'I2C'...",ESP-IDFでスレーブのレジスタアドレスを指定したreadのやり方,2021-05-05T11:21:01+09:00,https://qiita.com/badmintoncryer/items/752f8fe...,"{'description': '', 'facebook_id': '', 'follow...",None,None
4,<p>記事には書いたことなかったかも、と思い、取り急ぎメモしました。<br>\n二重だったら...,記事には書いたことなかったかも、と思い、取り急ぎメモしました。\n二重だったらごめんなさい。...,False,0,2021-05-05T11:18:24+09:00,None,746df2dcfcb5f46342d9,0,False,0,"[{'name': 'Excel', 'versions': []}, {'name': '...",小ネタ：年度とかの計算｜Power Query,2021-05-05T11:18:24+09:00,https://qiita.com/tanuki_phoenix/items/746df2d...,{'description': 'Power Queryの活用を進めています。Accessも...,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,<p>目的<br>\n動きそうだから動かしてみた<br>\n容量不足でほぼ何もできなかった。...,\n目的\n動きそうだから動かしてみた\n容量不足でほぼ何もできなかった。\n\nx mbe...,False,0,2021-05-05T00:58:29+09:00,None,3e5d7bab624051d69357,0,False,0,"[{'name': 'STM32L010', 'versions': []}]",STM32L010のシリアルでHello Worldと出力,2021-05-05T00:58:29+09:00,https://qiita.com/caa45040/items/3e5d7bab62405...,"{'description': '', 'facebook_id': '', 'follow...",None,None
96,"<p><a href=""https://qiita.com/fmax/items/4e9b0...","[Raspberry Pi Picoでモータを動かす②（ユニポーラ2相,1-2相励磁）](h...",False,0,2021-05-05T00:58:25+09:00,None,f4ae62f393bc4fc5b49a,0,False,0,"[{'name': 'RaspberryPiPico', 'versions': []}]",Raspberry Pi Picoでモータを動かす③（PIOで動かす①）,2021-05-05T00:58:25+09:00,https://qiita.com/fmax/items/f4ae62f393bc4fc5b49a,"{'description': '面白いものを作りたいが飽き性で中々作れないヒト。', 'f...",None,None
97,<p>確認は<code>Kotlin 1.4.32</code>で行いました。</p>\n\...,確認は`Kotlin 1.4.32`で行いました。\n\n# TL;DR\n- `KType...,False,0,2021-05-05T00:51:18+09:00,None,46270820077867b89cc9,0,False,0,"[{'name': 'Kotlin', 'versions': []}, {'name': ...",【Kotlin】unsigned integers/inline class引数のコンパイル...,2021-05-05T11:27:13+09:00,https://qiita.com/wrongwrong/items/46270820077...,{'description': 'リフレクションを用いたKotlin JVM向けのマッピング...,None,None
98,<p>現在チェックアウトしていないマージ済みのブランチを一括削除します。</p>\n\n<p...,現在チェックアウトしていないマージ済みのブランチを一括削除します。\n\nウェブを検索すると...,False,0,2021-05-05T00:46:09+09:00,None,949b58b47132f7199aab,0,False,0,"[{'name': 'Git', 'versions': []}]",Bash/Zsh/PowerShell 上の Git でマージ済みのブランチを一括削除,2021-05-05T00:50:38+09:00,https://qiita.com/watagashi/items/949b58b47132...,"{'description': '', 'facebook_id': 'watagashi'...",None,None


以上2つの理由により、パターン③のDataFrame作成方法をおすすめしています。

ただ分かりやすいだけでなく、データ取得の結果をそのままDataFrameに変換できるメリットもあるので、まずはこの方法をマスターしておけば問題ないと思います。

<br>

僕のYouTubeで紹介している[スクレイピング入門](https://www.youtube.com/playlist?list=PL4Y-mUWLK2t1LehwHVwAqxXTXw5xd-Yq8)でも、この方法でDataFrameを作成しています。

<br>

Pandasの扱いに慣れてきたら、他のDataFrame作成方法も習得して幅を広げていきましょう。

# まとめ

というわけで、今回はおすすめのDataFrame作成方法を紹介しました。

ネット上の記事を見ていると「DataFrameの作成はあまり使われない」と書かれていますが、決してそんなことはありません。

<br>

例えば株/為替のデータ分析をするとき、APIを使って金融データを取得したら、まずは加工しやすいようにDataFrameに落とし込みます。

今回のようにQiitaの記事情報を保存するにしても、DataFrameに落とし込んであげるのが吉です。

このように、DataFrameの作成もちゃんと使っていく文法なので、しっかりおさえておきましょう！

<br>

ただし、企業でおこなうデータ分析などは、CSVやExcelファイルからデータを読み込んだり、データベース/DWHからデータを抽出したりすることが大半です。

そのため、次回の講義ではCSVやExcelを読み込んで、DataFrameに落とし込む方法を紹介します。

<br>

文法としては難しいものではないので、この調子で頑張っていきましょう！


<hr>

また、講義の感想をメンションつけてツイートしていただけると嬉しいです！

Twitter : はやたす([@hayatasuuu](https://twitter.com/hayatasuuu) )